In [1]:
from planet4 import markings, dbscan, io

### Some regiondata to produce

In [2]:
from planet4 import region_data

In [4]:
dbscanner = dbscan.DBScanner()
for season in region_data.Inca.seasons:
    for image_name in season:
        dbscanner.cluster_image_name(image_name)

['PSP_002380_0985', 'PSP_002868_0985', 'PSP_003092_0985', 'PSP_003158_0985', 'PSP_003237_0985', 'PSP_003448_0985', 'PSP_003593_0985', 'PSP_003770_0815', 'PSP_003804_0985', 'PSP_003928_0815', 'PSP_004081_0985', 'PSP_004371_0985']
['ESP_013113_0985', 'ESP_013034_0985', 'ESP_012889_0985', 'ESP_012744_0985', 'ESP_012691_0985', 'ESP_012467_0985', 'ESP_012322_0985', 'ESP_012256_0985', 'ESP_011900_0985', 'ESP_011729_0985', 'ESP_011702_0985', 'ESP_011623_0985', 'ESP_011557_0985', 'ESP_011544_0985', 'ESP_011491_0985']
['ESP_022699_0985', 'ESP_021460_0985', 'ESP_020959_0985', 'ESP_020748_0985', 'ESP_020194_0985', 'ESP_020128_0985', 'ESP_020049_0985', 'ESP_021829_0985', 'ESP_021684_0985', 'ESP_021671_0985', 'ESP_021605_0985', 'ESP_021526_0985', 'ESP_020827_0985', 'ESP_020339_0985', 'ESP_020115_0985']
['ESP_030084_0985', 'ESP_030229_0985', 'ESP_029886_0985', 'ESP_029596_0985', 'ESP_029240_0985', 'ESP_029227_0985', 'ESP_029095_0985', 'ESP_028911_0985', 'ESP_028910_0985', 'ESP_028884_0985', 'ESP_028

In [ ]:
image_names = store.select_column('df', 'image_name').unique()

In [ ]:
from IPython.parallel import Client
client = Client()

In [ ]:
dview = client.direct_view()
lview = client.load_balanced_view()

In [ ]:
%%px
import pandas as pd
from planet4 import clustering, markings
from os.path import join as pjoin
import os
HOME = os.environ['HOME']

In [ ]:
def do_clustering(p4img, fans):
    if fans:
        reduced = clustering.perform_dbscan(p4img.get_fans(), fans=fans)
    else:
        reduced = clustering.perform_dbscan(p4img.get_blotches(), fans=fans)
    if reduced is None:
        return None
    series = [cluster.data for cluster in reduced]
    n_members = [cluster.n_members for cluster in reduced]
    df = pd.DataFrame(series)
    df['image_id'] = p4img.imgid
    df['n_members'] = n_members
    return df
    
def process_image_name(image_name):
    dirname = pjoin(HOME, 'data/planet4/reduced')
    blotchfname = pjoin(dirname, image_name+'_reduced_blotches.hdf')
    fanfname = pjoin(dirname, image_name+'_reduced_fans.hdf')
    if os.path.exists(blotchfname) and\
            os.path.exists(fanfname):
        return image_name+' already done.'
    data = pd.read_hdf(dbfile, 'df', where="image_name="+image_name)
    img_ids = data.image_id.unique()
    blotches = []
    fans = []
    for img_id in img_ids:
        p4img = markings.ImageID(img_id)
        blotches.append(do_clustering(p4img, fans=False))
        fans.append(do_clustering(p4img, fans=True))
    blotches = pd.concat(blotches, ignore_index=True)
    blotches.to_hdf(blotchfname, 'df')
    fans = pd.concat(fans, ignore_index=True)
    fans.to_hdf(fanfname, 'df')
    return image_name

In [ ]:
dview.push({'do_clustering':do_clustering,
            'dbfile':dbfile})

In [ ]:
result = lview.map_async(process_image_name, image_names)

In [ ]:
for res in result:
    print(res)

In [ ]:
import time
import sys
while not result.ready():
    print("{:.1f} %".format(100*result.progress/len(image_names)))
    sys.stdout.flush()
    time.sleep(30)

In [ ]:
reducedfiles = !ls ~/data/planet4/reduced
nooffiles = len(reducedfiles)
print("Produced", nooffiles, "files.")

In [ ]:
from planet4.get_data import is_catalog_production_good

In [ ]:
is_catalog_production_good()